In [3]:
import pandas as pd

In [4]:
ml_df =pd.read_parquet('ml_df-2.parquet')

In [16]:
ml_df.head()

,employee_id,year,month,vacation_days,vacations_cnt,start_min,vacation_flag,is_summer_month,department,department_size,...,has_irregular_schedule,extra_days_irregular,days_available,tenure_months,is_new_employee,tenure_was_invalid,vac_flag_lag1,vac_days_lag1,vac_flag_last3,vac_flag_last6
0,EMP_000031DF,2020,8,0,0,NaT,0,1,DEPT_E8D5EC,16.0,...,0.0,0.0,28.0,37.023653,0.0,0.0,0,0,0,0
1,EMP_000031DF,2020,9,0,0,NaT,0,0,DEPT_E8D5EC,16.0,...,0.0,0.0,28.0,37.023653,0.0,0.0,0,0,0,0
2,EMP_000031DF,2020,10,0,0,NaT,0,0,DEPT_E8D5EC,16.0,...,0.0,0.0,28.0,37.023653,0.0,0.0,0,0,0,0
3,EMP_000031DF,2020,11,0,0,NaT,0,0,DEPT_E8D5EC,16.0,...,0.0,0.0,28.0,37.023653,0.0,0.0,0,0,0,0
4,EMP_000031DF,2020,12,0,0,NaT,0,0,DEPT_E8D5EC,16.0,...,0.0,0.0,28.0,37.023653,0.0,0.0,0,0,0,0


In [4]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8213855 entries, 0 to 8213854
Data columns (total 31 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   employee_id             object        
 1   year                    int32         
 2   month                   int32         
 3   vacation_days           int16         
 4   vacations_cnt           int8          
 5   start_min               datetime64[ns]
 6   vacation_flag           int8          
 7   is_summer_month         int8          
 8   department              object        
 9   department_size         float64       
 10  region                  object        
 11  is_manager              float64       
 12  manager_avoids_summer   float64       
 13  is_north_region         float64       
 14  north_bonus_days        float64       
 15  age_years               float64       
 16  birth_month             float64       
 17  is_married              float64       
 18  ch

In [5]:
import pandas as pd
import numpy as np

ml_df = ml_df.sort_values(["employee_id", "year", "month"]).copy()
ml_df["ym"] = pd.PeriodIndex(year=ml_df["year"], month=ml_df["month"], freq="M")


/tmp/ipython-input-598020791.py:5: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  ml_df["ym"] = pd.PeriodIndex(year=ml_df["year"], month=ml_df["month"], freq="M")


In [6]:
df_test = ml_df[ml_df["year"] == 2025].copy()
df_cv = ml_df[(ml_df["year"] >= 2022) & (ml_df["year"] <= 2024)].copy()

print(df_cv.shape, df_test.shape)
print("target mean cv:", df_cv["vacation_flag"].mean())
print("target mean test:", df_test["vacation_flag"].mean())


(4549212, 32) (1516404, 32)
target mean cv: 0.15605164147109432
target mean test: 0.1380469848404515


In [7]:
dropcols = ["employee_id", "vacation_flag", "vacation_days", "vacations_cnt", "start_min"]
features = [c for c in df_cv.columns if c not in dropcols + ["ym"]]

# категориальные
catcols = [c for c in features if df_cv[c].dtype == "object"]

# бинарные
binarycols = [
    "is_manager", "manager_avoids_summer", "is_north_region", "is_married",
    "has_school_children", "has_irregular_schedule", "is_new_employee",
    "tenure_was_invalid"
]
binarycols = [c for c in binarycols if c in features]

def fill_features(df, train_stats=None):
    df = df.copy()

    # категории
    for c in catcols:
        df[c] = df[c].astype("string").fillna("missing")

    # бинарные
    for c in binarycols:
        df[c] = df[c].fillna(0).astype("int8")

    # числовые — медианы считаем по train и применяем в valid/test
    numcols = [c for c in features if (c not in catcols) and (c not in binarycols)]
    if train_stats is None:
        train_stats = {c: df[c].median() for c in numcols}
    for c in numcols:
        df[c] = df[c].fillna(train_stats[c])

    return df, train_stats


In [8]:
pip install catboost

In [9]:
import gc
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, average_precision_score


In [22]:
def make_time_folds(df, val_window_months=3):
    months = sorted(df["ym"].unique())
    folds = []

    for i in range(len(months) - val_window_months):
        start = months[i]
        end = months[i + val_window_months - 1]

        train_mask = df["ym"] < start
        valid_mask = (df["ym"] >= start) & (df["ym"] <= end)

        if train_mask.sum() == 0 or valid_mask.sum() == 0:
            continue

        folds.append((start, end, train_mask, valid_mask))

    return folds


In [23]:
def run_cv(sample_train_rows=600000):

    folds = make_time_folds(df_cv, val_window_months=3)
    results = []

    for (start, end, train_mask, valid_mask) in folds:

        train_part = df_cv.loc[train_mask, features + ["vacation_flag"]]
        valid_part = df_cv.loc[valid_mask, features + ["vacation_flag"]]

        # Берём сэмпл чтобы не падала память
        if len(train_part) > sample_train_rows:
            train_part = train_part.sample(sample_train_rows, random_state=42)

        Xtrain = train_part[features]
        ytrain = train_part["vacation_flag"].astype("int8")

        Xvalid = valid_part[features]
        yvalid = valid_part["vacation_flag"].astype("int8")

        model = CatBoostClassifier(
            iterations=150,
            depth=5,
            learning_rate=0.1,
            loss_function="Logloss",
            eval_metric="AUC",
            auto_class_weights="Balanced",
            random_seed=42,
            verbose=False
        )

        model.fit(
            Xtrain,
            ytrain,
            eval_set=(Xvalid, yvalid),
            cat_features=catcols,
            use_best_model=True
        )

        valid_proba = model.predict_proba(Xvalid)[:, 1]

        roc = roc_auc_score(yvalid, valid_proba)
        pr = average_precision_score(yvalid, valid_proba)

        print("Fold", start, "to", end,
              "| ROC AUC:", round(roc,4),
              "| PR AUC:", round(pr,4))

        results.append(roc)

        # очистка памяти
        del train_part, valid_part, Xtrain, Xvalid, model
        gc.collect()

    print("\nСредний ROC AUC:", np.mean(results))


In [10]:
sample_part = df_cv.sample(100000, random_state=42)

Xcheck = sample_part[features].copy()

print(Xcheck.isna().sum()[Xcheck.isna().sum() > 0].head(20))


department                6747
department_size           6747
region                    6747
is_manager                6747
manager_avoids_summer     6747
is_north_region           6747
north_bonus_days          6747
age_years                 6747
birth_month               6747
is_married                6747
children_count            6747
school_children_count     6747
has_school_children       6747
has_irregular_schedule    6747
extra_days_irregular      6747
days_available            6747
tenure_months             6747
is_new_employee           6747
tenure_was_invalid        6747
dtype: int64


In [11]:
import numpy as np

# --- 1. категории ---
for col in ["department", "region"]:
    if col in Xcheck.columns:
        Xcheck[col] = Xcheck[col].astype("string").fillna("missing")

# --- 2. бинарные ---
binary_cols = [
    "is_manager",
    "manager_avoids_summer",
    "is_north_region",
    "is_married",
    "has_school_children",
    "has_irregular_schedule",
    "is_new_employee",
    "tenure_was_invalid"
]

for col in binary_cols:
    if col in Xcheck.columns:
        Xcheck[col] = Xcheck[col].fillna(0).astype("int8")

# --- 3. числовые ---
num_cols = [c for c in Xcheck.columns if c not in binary_cols + ["department", "region"]]

medians = Xcheck[num_cols].median()

Xcheck[num_cols] = Xcheck[num_cols].fillna(medians)

print("Осталось NaN:", Xcheck.isna().sum().sum())


Осталось NaN: 0


In [12]:
import pandas as pd
import numpy as np
import gc

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

ml_df = ml_df.sort_values(["employee_id", "year", "month"]).copy()
ml_df["ym"] = pd.PeriodIndex(year=ml_df["year"], month=ml_df["month"], freq="M")

# CV делаем на 2022-2024, тест 2025 держим отдельно
df_cv = ml_df[(ml_df["year"] >= 2022) & (ml_df["year"] <= 2024)].copy()
df_test = ml_df[ml_df["year"] == 2025].copy()

print(df_cv.shape, df_test.shape)
print("target mean cv:", float(df_cv["vacation_flag"].mean()))
print("target mean test:", float(df_test["vacation_flag"].mean()))


/tmp/ipython-input-2055628557.py:9: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  ml_df["ym"] = pd.PeriodIndex(year=ml_df["year"], month=ml_df["month"], freq="M")


(4549212, 32) (1516404, 32)
target mean cv: 0.15605164147109432
target mean test: 0.1380469848404515


In [13]:
dropcols = ["employee_id", "vacation_flag", "vacation_days", "vacations_cnt", "start_min", "ym"]
features = [c for c in df_cv.columns if c not in dropcols]

catcols = [c for c in features if df_cv[c].dtype == "object"]

binarycols = [
    "is_manager", "manager_avoids_summer", "is_north_region", "is_married",
    "has_school_children", "has_irregular_schedule", "is_new_employee",
    "tenure_was_invalid"
]
binarycols = [c for c in binarycols if c in features]

print("features:", len(features))
print("catcols:", catcols)
print("binarycols:", binarycols)


features: 26
catcols: ['department', 'region']
binarycols: ['is_manager', 'manager_avoids_summer', 'is_north_region', 'is_married', 'has_school_children', 'has_irregular_schedule', 'is_new_employee', 'tenure_was_invalid']


In [14]:
def make_time_folds(df, val_window_months=3):
    months = sorted(df["ym"].unique())
    folds = []
    for i in range(len(months) - val_window_months):
        start = months[i]
        end = months[i + val_window_months - 1]
        train_mask = df["ym"] < start
        valid_mask = (df["ym"] >= start) & (df["ym"] <= end)
        if train_mask.sum() == 0 or valid_mask.sum() == 0:
            continue
        folds.append((start, end, train_mask, valid_mask))
    return folds

def fill_block(Xtrain, Xvalid):
    # копии, чтобы не было SettingWithCopyWarning
    Xtrain = Xtrain.copy()
    Xvalid = Xvalid.copy()

    # категории
    for c in catcols:
        Xtrain[c] = Xtrain[c].astype("string").fillna("missing")
        Xvalid[c] = Xvalid[c].astype("string").fillna("missing")

    # бинарные
    for c in binarycols:
        Xtrain[c] = Xtrain[c].fillna(0).astype("int8")
        Xvalid[c] = Xvalid[c].fillna(0).astype("int8")

    # числовые: None -> NaN, затем медианы train
    Xtrain = Xtrain.replace({None: np.nan})
    Xvalid = Xvalid.replace({None: np.nan})

    numcols = [c for c in features if (c not in catcols) and (c not in binarycols)]
    med = Xtrain[numcols].median()
    Xtrain[numcols] = Xtrain[numcols].fillna(med)
    Xvalid[numcols] = Xvalid[numcols].fillna(med)

    return Xtrain, Xvalid


In [15]:
def run_cv(sample_train_rows=600000, val_window_months=3, max_folds=8):

    folds = make_time_folds(df_cv, val_window_months=val_window_months)

    rocs = []
    prs = []

    for fold_index, (start, end, train_mask, valid_mask) in enumerate(folds):

        # ограничим количество фолдов, чтобы быстро проверить (потом можно убрать)
        if max_folds is not None and fold_index >= max_folds:
            break

        train_part = df_cv.loc[train_mask, features + ["vacation_flag"]]
        valid_part = df_cv.loc[valid_mask, features + ["vacation_flag"]]

        # сэмпл train
        if len(train_part) > sample_train_rows:
            train_part = train_part.sample(sample_train_rows, random_state=42)

        Xtrain_raw = train_part[features]
        ytrain = train_part["vacation_flag"].astype("int8")

        Xvalid_raw = valid_part[features]
        yvalid = valid_part["vacation_flag"].astype("int8")

        Xtrain, Xvalid = fill_block(Xtrain_raw, Xvalid_raw)

        model = CatBoostClassifier(
            iterations=150,
            depth=5,
            learning_rate=0.1,
            loss_function="Logloss",
            eval_metric="AUC",
            auto_class_weights="Balanced",
            random_seed=42,
            verbose=False
        )

        model.fit(
            Xtrain, ytrain,
            eval_set=(Xvalid, yvalid),
            cat_features=catcols,
            use_best_model=True
        )

        proba = model.predict_proba(Xvalid)[:, 1]
        roc = roc_auc_score(yvalid, proba)
        pr = average_precision_score(yvalid, proba)

        print("Fold", fold_index + 1, "|", start, "to", end, "| ROC AUC:", round(roc,4), "| PR AUC:", round(pr,4))

        rocs.append(roc)
        prs.append(pr)

        del train_part, valid_part, Xtrain_raw, Xvalid_raw, Xtrain, Xvalid, model
        gc.collect()

    print("\nMEAN ROC AUC:", float(np.mean(rocs)))
    print("MEAN PR AUC :", float(np.mean(prs)))
    return rocs, prs


In [16]:
rocs, prs = run_cv(sample_train_rows=600000, val_window_months=3, max_folds=8)


Fold 1 | 2022-02 to 2022-04 | ROC AUC: 0.578 | PR AUC: 0.1342
Fold 2 | 2022-03 to 2022-05 | ROC AUC: 0.5859 | PR AUC: 0.1933
Fold 3 | 2022-04 to 2022-06 | ROC AUC: 0.5994 | PR AUC: 0.2574
Fold 4 | 2022-05 to 2022-07 | ROC AUC: 0.6487 | PR AUC: 0.3316
Fold 5 | 2022-06 to 2022-08 | ROC AUC: 0.6645 | PR AUC: 0.387
Fold 6 | 2022-07 to 2022-09 | ROC AUC: 0.6857 | PR AUC: 0.4067
Fold 7 | 2022-08 to 2022-10 | ROC AUC: 0.6885 | PR AUC: 0.3953
Fold 8 | 2022-09 to 2022-11 | ROC AUC: 0.6737 | PR AUC: 0.3285

MEAN ROC AUC: 0.6405556609469145
MEAN PR AUC : 0.30426372711235933
